In [2]:
from pulp import *
import csv
import numpy as np
import pandas as pd

In [115]:
final_df.columns = ['name', 'salary', 'avg_points', 'odds']
final_df['odds'] = final_df['odds']/ 10
os.chdir('/home/valesco/Documents/')
final_df.to_csv('British.csv')

final_df.head()

/home/valesco/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,name,salary,avg_points,odds
0,Jason Day,12300,82.875,8.0
1,Dustin Johnson,12000,91.643,8.0
2,Rory McIlroy,11900,83.867,9.0
3,Jordan Spieth,11700,81.633,10.0
4,Adam Scott,10600,84.231,22.0


In [99]:
in_file = "British.csv"
reader = csv.DictReader(open(in_file, 'r'))
player_dict = []
for i , row in enumerate(reader):
    t = {k: v for k, v in row.items()}
    t['id'] = i
    player_dict.append(t)
    
# Create an empty LP
IP = LpProblem("Golf Optimization", LpMinimize)

# Create a list of the teams and player IDs
teams = [i for i in range(10)]
players = [player['id'] for player in player_dict]

# Create binary decision variables for which player ends up on which team
possible_assignments = LpVariable.dicts("x", (teams, players), 0, 1, LpBinary)

# Add in the objective function
IP += sum([float(player['odds']) * possible_assignments[team][player['id']] for player in player_dict for team in teams])
#IP += 

for team in teams:
    IP += lpSum([possible_assignments[team][player['id']] for player in player_dict]) == 6, '6 players per team '+str(team)
    IP += sum([float(player['salary']) * possible_assignments[team][player['id']] for player in player_dict]) <= 50000
    IP += sum([float(player['odds']) < 15 for player in player_dict]) > 0, '1 favorite per team'
    #IP += lpSum([possible_assignments[t][p['id']] for p in d if p['Group'] == 'A']) == 1, '1 A per team '+str(t)
    #IP += lpSum([possible_assignments[t][p['id']] for p in d if p['Group'] == 'B']) == 2, '2 B per team '+str(t)
    #IP += lpSum([possible_assignments[t][p['id']] for p in d if p['Group'] == 'C']) == 3, '3 C per team '+str(t)
    #IP += lpSum([possible_assignments[t][p['id']] for p in d if p['Group'] == 'D']) == 2, '2 D per team '+str(t)
    
    
    
    
# Write out the LP formulation
IP.writeLP('simple optimization.lp')

In [100]:
# Solve the LP
IP.solve()

# Print out the LP solution status
print("Status:", LpStatus[IP.status])

Status: Optimal


In [106]:
# Print out some results
print("The players chosen for team 1 are:")

opt_salary = []
opt_odds = []
for player in player_dict:
    if possible_assignments[1][player['id']].value() == 1.0:
        print(player['id'])
        opt_odds.append(float(player['odds']))
        opt_salary.append(int(player['salary']))

print('Total Salary - %i, Avg odds to win - %i' % (sum(opt_salary), np.mean(opt_odds)))
        

The players chosen for team 1 are:
32
114
115
116
119
120
Total Salary - 34800, Avg odds to win - 57916


In [5]:
#Cat food simplified Model
#Create the problem variable to contain the problem data
prob = LpProblem('The Whiskas Problem', LpMinimize)

#Create 2 variables Beef and Chicken with lower limit of 0
x1 = LpVariable('Chick Percent', 0, None, LpInteger)
x2 = LpVariable('Beef Percent', 0)

#Create the objective of the optimization
prob += 0.013 * x1 + 0.008 * x2, "Total Cost of Ingredients per can"

#Create 5 constraints
prob += x1 + x2 == 100, 'Percentage Sum'
prob += 0.100 * x1 + 0.200 * x2 >= 8.0, 'Protein Requirement'
prob += 0.080 * x1 + 0.100 * x2 >= 6.0, 'Fat Requirement'
prob += 0.001 * x1 + 0.005 * x2 <= 2.0, 'Fiber Requirement'
prob += 0.002 * x1 + 0.005 * x2 <= 0.4, 'Salt Rquirement'

In [6]:
# Solve problem
prob.solve()
print('Status: ', LpStatus[prob.status])

Status:  Optimal


In [7]:
for v in prob.variables():
    print(v.name, ' = ', v.varValue)

Beef_Percent  =  66.0
Chick_Percent  =  34.0


In [8]:
# The optimized objective function value is printed to the screen
print('Total Cost of Ingredients per can = ', value(prob.objective))

Total Cost of Ingredients per can =  0.97


In [9]:
#Cat food full version
# Create a list of ingredients
Ingredients = ['Chicken', 'Beef', 'Mutton', 'Rice', 'Wheat', 'Gel']

# Dict of the cost of each ingredient
costs = {'Chicken': 0.013,
         'Beef': 0.008,
         'Mutton': 0.010,
         'Rice': 0.002,
         'Wheat': 0.005,
         'Gel': 0.001}

# Dict of protein percent
proteinPercent = {'Chicken': 0.100,
                  'Beef': 0.200,
                  'Mutton': 0.150,
                  'Rice': 0.000,
                  'Wheat': 0.040,
                  'Gel': 0.000}

# A dictionary of the fat percent
fatPercent = {'Chicken': 0.080,
              'Beef': 0.100,
              'Mutton': 0.110,
              'Rice': 0.010,
              'Wheat': 0.010,
              'Gel': 0.000}

# Dict of fiber
fiberPercent = {'Chicken': 0.001,
                'Beef': 0.005,
                'Mutton': 0.003,
                'Rice': 0.100,
                'Wheat': 0.150,
                'Gel': 0.000}

# Dict of salt percent
saltPercent = {'Chicken': 0.002,
               'Beef': 0.005,
               'Mutton': 0.007,
               'Rice': 0.002,
               'Wheat': 0.008,
               'Gel': 0.000}

In [10]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem('The Whiskas Problem', LpMinimize)

# Dict called ingredient vars to contain reference variables
ingredient_vars = LpVariable.dicts("Ingr",Ingredients,0)

# The objective funct is added to prob first
prob += lpSum([costs[i] * ingredient_vars[i] for i in Ingredients]), 'Total Cost of Ingredients per can'

# The five constraints are added to 'prob'
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fiberPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"

In [11]:
# Solve problem
prob.solve()
print('Status: ', LpStatus[prob.status])

Status:  Optimal


In [12]:
for v in prob.variables():
    print(v.name, ' = ', v.varValue)

Ingr_Beef  =  60.0
Ingr_Chicken  =  0.0
Ingr_Gel  =  40.0
Ingr_Mutton  =  0.0
Ingr_Rice  =  0.0
Ingr_Wheat  =  0.0


In [13]:
# The optimized objective function value is printed to the screen
print('Total Cost of Ingredients per can = ', value(prob.objective))

Total Cost of Ingredients per can =  0.52


In [116]:
in_file = "British.csv"
reader = csv.DictReader(open(in_file, 'r'))
player_dict = []
for i , row in enumerate(reader):
    team = {k: v for k, v in row.items()}
    team['id'] = i
    player_dict.append(team)
    

In [144]:
teams = [i for i in range(10)]
players = [player['id'] for player in player_dict]


golf = LpProblem('US Open', LpMinimize)

possible_assignments = LpVariable.dicts("possible", (teams, players), 0, 1, LpBinary)

#objective
golf += sum([float(player['odds']) * possible_assignments[team][player['id']] for player in player_dict for team in teams])

for team in teams:
    golf += lpSum([possible_assignments[team][player['id']] for player in player_dict]) == 6, '6 players per team '+str(team)
    golf += sum([float(player['salary']) * possible_assignments[team][player['id']] for player in player_dict]) <= 50000
    golf += lpSum([possible_assignments[team][player['id']] for player in player_dict if float(player['odds']) < 15]) >= 1
    #golf += lpSum([possible_assignments[team][player['id']] for player in player_dict if float(player['Odds']) < 20]) >= 2
    golf += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['name'] == 'Zach Johnson']) == 0
    golf += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['name'] == 'Bubba Watson']) == 0
    golf += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['name'] == 'Jon Rahm']) == 1
    golf += lpSum([possible_assignments[team][player['id']] for player in player_dict if player['name'] == 'Phil Mickelson']) == 0

In [145]:
golf.writeLP('masters_optimization.lp')
golf.solve()
print('Status: ', LpStatus[golf.status])


Status:  Optimal


In [146]:
# Print out some results
print("The players chosen for team 1 are:")

opt_salary = []
opt_odds = []
for player in player_dict:
    if possible_assignments[1][player['id']].value() == 1.0:
        print(player['name']+ '- odds: ' + player['odds'] + ' salary: ' + player['salary'])
        opt_odds.append(float(player['odds']))
        opt_salary.append(int(player['salary']))

print('Total Salary - %i, Avg odds to win - %i' % (sum(opt_salary), np.mean(opt_odds)))
        

The players chosen for team 1 are:
Jordan Spieth- odds: 10.0 salary: 11700
Shane Lowry- odds: 45.0 salary: 8800
Patrick Reed- odds: 55.0 salary: 7900
Charl Schwartzel- odds: 66.0 salary: 7800
Andy Sullivan- odds: 80.0 salary: 7200
Jon Rahm- odds: 150.0 salary: 6600
Total Salary - 50000, Avg odds to win - 67


In [ ]:
for team in teams:   
    opt_salary = []
    opt_odds = []
    for player in player_dict:
        if possible_assignments[1][player['id']].value() == 1.0:
            print(player['Name']+ '- odds: ' + player['Odds'] + ' Salary: ' + player['Salary'])
            opt_odds.append(float(player['Odds']))
            opt_salary.append(int(player['Salary']))

    print('Total Salary - %i, Avg odds to win - %i' % (sum(opt_salary), np.mean(opt_odds)))